# HTML Tables and Table Formatting


## Creating a HTML Table from pandas.DataFrame
The following is hopefully sufficient for most applications (feedback welcome!):
```
from pybloqs import Blockb = Block(df)
```

When called only with DataFrame as parameter, a set of default formatters is applied: 
```
table_block = Block(df, formatters=None, use_default_formatters=True)
```

In [3]:
import pybloqs as abl
from pybloqs import Block
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.rand(4,4), index=['a','b','c','d'], columns = ['aa','bb','cc','dd'])
df.index.name = 'ABC'
table_block = Block(df)
output = table_block.render_html()


# Displaying pybloqs out in jupyter requires rendering html output
from IPython.core.display import display, HTML
display(HTML(output))

ABC,aa,bb,cc,dd
a,0.49,0.36,0.97,0.77
b,0.07,0.62,0.07,0.56
c,0.39,0.11,0.75,0.37
d,0.34,0.20,0.71,0.95


NB: The grid between cells is from jupyter default CSS. It will not show if the block is displayed with b.show() .

## Formatting Tables with Table Formatters
Formatters are functions which add a single specific formatting aspect (e.g. bold, font-size, alignment, multi-index display). Formatters can be stacked together as a list to produce desired layout. The list is then passed to ```HTMLJinjaTableBlock```.

Use of default formatters can be disabled completely. Additional formatters can be used on top or instead of default formatters.
Formatters change appearance by modifying cell values and adding CSS styles.

'Exotic' formatters, which are used only in a single context, can be defined locally.
A set of general use formatters can be found in pybloqs.block.table_formatters.

All formatters take the following parameters:
```
:rows List of rows, where formatter should be applied
:columns List of columns, where formatter should be applied
:apply_to_header_and_index True/False, if set to True, formatter will be applied to all index and header cells
```
If rows and columns are both ```None```, formatter is applied to all cells in table.

An example:

In [4]:
import pybloqs.block.table_formatters as tf
table_block = Block(df)
table_block_raw = Block(df, use_default_formatters=False)

fmt_pct = tf.FmtPercent(1, columns=['bb','cc'], apply_to_header_and_index=False)
fmt_totals = tf.FmtAppendTotalsRow(total_columns=['aa','dd'])
fmt_highlight = tf.FmtHighlightText(columns=['bb'], rows=['d'], apply_to_header_and_index=False)
formatters=[fmt_pct,fmt_totals, fmt_highlight]
table_block_additional_formatters = Block(df, formatters=formatters)

fmt_mult = tf.FmtMultiplyCellValue(1e6, '')
fmt_sep = tf.FmtThousandSeparator()
formatters=[fmt_mult, fmt_sep]
table_block_new_formatters = Block(df, formatters=formatters, use_default_formatters=False)

row1 = abl.HStack([table_block, table_block_raw])
row2 = abl.HStack([table_block_additional_formatters, table_block_new_formatters])
all_tables = abl.VStack([row1,row2])

display(HTML(all_tables.render_html()))


ABC,aa,bb,cc,dd
a,0.49,0.36,0.97,0.77
b,0.07,0.62,0.07,0.56
c,0.39,0.11,0.75,0.37
d,0.34,0.20,0.71,0.95
ABC,aa,bb,cc,dd
a,0.491425968128,0.355021291548,0.969661643322,0.771977085029
b,0.066542981958,0.622449011817,0.0709732127762,0.557307898499
c,0.392367746736,0.105056261555,0.745123712728,0.372812760257
d,0.337316831943,0.198479494974,0.708809001578,0.952886753603
ABC,aa,bb,cc,dd


### General formatters
The following formatters handle miscalleneous general tasks
#### Replace NaN
```
FmtReplaceNaN(value='')
```
Replaces all np.nan values in specified range with provided ```value```. By default uses empty string.

#### FmtAlignCellContents
```
FmtAlignCellContents(alignment='center')
```
Aligns content inside cells within specified range. Valid values for ```alignment``` are ```left|right|center|justify|initial|inherit``` (anything that the CSS tag ```text-align``` understands).

#### FmtAlignTable
```
FmtAlignTable(alignment)
```
Aligns the entire table relative to its entironment. Valid values for alignment are ```center```, ```right```, ```left```.

#### FmtHeader
```
FmtHeader(fixed_width='100%', index_width=None, column_width=None, rotate_deg=0,top_padding=None, no_wrap=True)
```
Creates a table with fixed-width columns. 
```
:fixed_width Total width of table
:index_width Fixed width of index column
:column_width Fixed width of all other columns
:rotate_deg Value in degrees by which to rotate table header cells
:top_padding: additional vertical space above table, may be necessary when using rotated headers
:no_wrap True/False, disables line-breaks within header cell when set to True
```
An example (NB, jypiter ignores top-padding, which otherwise works in direkt browser display and PDF output):

In [7]:
fmt_header = tf.FmtHeader(fixed_width='20cm',index_width='30%', top_padding='3cm', rotate_deg=30)
table_block = Block(df, formatters=[fmt_header])
display(HTML(table_block.render_html()))

a,0.49,0.36,0.97,0.77
b,0.07,0.62,0.07,0.56
c,0.39,0.11,0.75,0.37
d,0.34,0.20,0.71,0.95


#### FmtStripeBackground
```
FmtStripeBackground(first_color=colors.LIGHT_GREY, second_color=colors.WHITE, header_color=colors.WHITE,
```
Creates a repeating color patters in the background of the specified cells.
```
:first_color CSS color, for odd row numbers
:second_color CSS color, for even row numbers
:header_color CSS color applied to header row
```
#### FmtAddCellPadding
```
FmtAddCellPadding(left=None, right=None, top=None, bottom=None, length_unit='px')
```
Add space on sides of selected cells.
#### FmtAppendTotalsRow
```
FmtAppendTotalsRow(row_name='Total', operator=OP_SUM, bold=True, background_color=colors.LIGHT_GREY, font_color=None, total_columns=None)
```
Adds a line at the end of the table filled with values computed columnwise. For an example, see section [Formatting Tables with Table Formatters]
```
:row_name Label for additional row shown in index
:operator Computational operation to perform on columns, e.g. tf.OP_SUM, tf.OP_MEAN, tf.OP_NONE
:total_columns Names of columns to apply operator to. If None, operator is applied to all columns.
:bold True/False, applied bold font-style to cells in added row
:font_color CSS color for cell text in added row
:background_color CSS color for cell background in added row
```
#### FmtHideRows
```
FmtHideCells(rows=None, columns=None)
```
Hides cells in the intersection of rows and columns list. If only rows or columns is specified and the other is left None, the entire row or column is hidden.
#### FmtPageBreak
```
FmtPageBreak(no_break=True, repeat_header=True)
```
Defines table behaviour around page-breaks. Please note that Webkit-based browsers (Chrome, Safari and wkhtmltopdf as well) do not handle the ```repeat-header``` property properly, especially when headers are rotated. This bug is reported and not resolved since 6 years. Functionality in Firefox is fine, including rotated headers.
```
:no_break True/False, avoids splitting table on page break
:repeat_header True/False, when table is split accross page, repeat header on the next page
```

### Displaying text
The following formatters modify the display of both text and numbers.
#### FmtFontsize
```
FmtFontsize(fontsize, format='px')
```
Sets the font-size property of specified cells. A nice property is ```vw```, which gives the font-size as percent of viewport-width. This will scale the font with the witdh of the page and is thus suited for wide tables which should still look fine (but small) when output as PDF.

#### FmtHighlightText
```
FmtHighlightText(bold=True, italic=True, font_color=colors.BLUE)
```
Sets various properties of character display.
```
:bold True/False, sets bold font-style
:italic True/False, sets italic font-style
:font_color CSS color, sets text color
```
#### FmtHighlightBackground
```
FmtHighlightBackground(color=colors.RED)
```
Sets the background color of specified cells.

#### FmtBold
```
FmtBold()
```
Sets font-style bold in specified cells.

### Displaying numbers
The following formatters only apply to numbers. Please note that some convert the number to a string when applied.

#### FmtNumbers, FmtDecimals, FmtThousandSeparator, FmtPercent
E.g.
```
FmtPercent(n_decimals)
```
If cell content is a number, it is changed to a string with approriate formatting, e.g. number of decimals (FmtDecimals), with a comma as thousands separator (FmtThousandSeparator), or as percent with a trailing '%' sign(FmtPercent).
```
FmtNumbers(fmt_string)
```
is the general purpose formatting class, which accets any formatting string. For more information about formatting strings, see https://pyformat.info/

#### FmtMultiplyCellValue, FmtValueToMillion, FmtValueToPercent, FmtValueToBps
E.g.
```
FmtValueToPercent(suffix='%')
```
Multiplies number in cell by a given factor, thus keeping is a number. A suffix an be added to the columns header. This is useful for tables, which all contain percentage values and where a '%' sign after each value is undesireable.
```
FmtMultiplyCellValue(d, suffix)
```
is the general purpose function, multiplying by any given factor.

### Heatmaps
The table formatting has a very flexible heatmap formatter.
#### FmtHeatmap
```
FmtHeatmap(min_color=colors.HEATMAP_RED, max_color=colors.HEATMAP_GREEN, threshold=0.,axis=None)
```
Creates a heatmap in the intersection of specified columns and rows.
```
:min_color CSS color, which is the color applied as background-color at the minimum negative value
:max_color CSS color, which is the color applied as background-color at the maximum positive value
:threshold specifies an interval around 0, in which no heatmapping is applied
:axis Number, either 0 (horizontal), or 1 (vertical) or None. If None, heatmap is applied over all selected cells. If set to a number, heatmap is applied column-wise or row-wise reprectively.
```

In [8]:
import string
# Create DataFrame
df_size = 15
index = [c for c in string.ascii_lowercase[:df_size]]
columns = [c+c for c in string.ascii_lowercase[:df_size]]
df = pd.DataFrame(np.random.rand(df_size,df_size), index=index, columns=columns)

# Specify heatmap formatters
# All values in range
fmt_heatmap1 = tf.FmtHeatmap(rows=index[10:16],columns=columns[:5])
# By row
fmt_heatmap2 = tf.FmtHeatmap(rows=index[:3], axis=0, max_color=(255,0,255))
# By column
fmt_heatmap3 = tf.FmtHeatmap(rows=index[5:],columns=columns[10:], axis=1, max_color=(255,255,0))

formatters =[fmt_heatmap1, fmt_heatmap2, fmt_heatmap3]

# Display
table_block = Block(df, formatters=formatters)
display(HTML(table_block.render_html()))

,aa,bb,cc,dd,ee,ff,gg,hh,ii,jj,kk,ll,mm,nn,oo
a,0.14,0.58,0.77,0.48,0.11,0.54,0.13,0.05,0.85,0.66,0.39,0.99,0.89,0.72,0.53
b,0.74,0.97,0.19,0.99,0.91,0.10,0.16,0.06,0.49,0.78,0.92,0.80,0.41,0.98,0.38
c,0.07,0.14,0.42,0.36,0.91,0.97,0.08,0.02,0.31,0.75,0.62,0.59,0.97,0.24,0.03
d,0.55,0.45,0.27,0.29,0.22,0.49,0.06,0.25,0.98,0.93,0.07,0.26,0.85,0.52,0.84
e,0.91,0.40,0.41,0.65,0.54,0.32,1.00,0.29,0.59,0.72,0.01,0.94,0.39,0.33,0.41
f,0.02,0.29,0.64,0.75,0.60,0.48,0.76,0.97,0.64,0.13,0.95,0.95,0.94,0.63,0.44
g,0.27,0.53,0.69,0.90,0.66,0.32,0.04,0.30,0.07,0.59,0.10,0.80,0.34,0.27,0.06
h,0.02,0.72,0.20,0.76,0.39,0.40,0.90,0.99,0.58,0.54,0.91,0.88,0.95,0.85,0.69
i,0.00,0.93,0.67,0.17,0.73,0.65,0.50,0.51,0.97,0.44,0.73,0.61,0.56,0.66,0.44
j,0.06,0.26,0.07,0.72,0.51,0.55,0.92,0.49,0.55,0.55,0.65,0.43,0.05,0.38,0.64


### Multi-index tables
Multi-index dataframes can be expanded to sinlesimple index dataframes with special formatting applied.
#### FmtExpandMultiIndex
```
FmtExpandMultiIndex(total_columns=None, operator=OP_SUM, bold=True, indent_px=20, hline_color=colors.DARK_BLUE, level_background_colors=None)
```
See example below. Can handle non-unique indices.
```
:total_columns List of columns on which to apply operator at higher index levels
:operator Computational operation to perform on columns, e.g. tf.OP_SUM, tf.OP_MEAN, tf.OP_NONE
:bold True/False, changes higher-level font-style to bold
:index_px Indentation space per level in pixels
:hline_color CSS color, sets the color of the horizontal line separating higher-level rows
:level_background_colors List of CSS colors with size equal to number of index-levels, background_color applied in each index-level row
```

In [9]:
def make_multiindex_table():
    fmt = tf.FmtExpandMultiIndex()
    idx = np.array([['a', 'a', 'b', 'b'], ['aa', 'ab', 'ba', 'bb']])
    idx_tuples = list(zip(*idx))
    multi_index = pd.MultiIndex.from_tuples(idx_tuples, names=['a-level', 'aa-level'])
    columns = ['column0', 'column1', 'column2']
    data = pd.DataFrame(np.arange(12, dtype=float).reshape(4, 3), index=multi_index, columns=columns)
    return data

mi_df = make_multiindex_table()
print mi_df

fmt_multiindex = tf.FmtExpandMultiIndex(operator=tf.OP_SUM)
table_block = Block(mi_df, formatters=[fmt_multiindex], use_default_formatters=False)
display(HTML(table_block.render_html()))

                  column0  column1  column2
a-level aa-level                           
a       aa              0        1        2
        ab              3        4        5
b       ba              6        7        8
        bb              9       10       11


,column0,column1,column2
a,3.0,5.0,7.0
aa,0.0,1.0,2.0
ab,3.0,4.0,5.0
b,15.0,17.0,19.0
ba,6.0,7.0,8.0
bb,9.0,10.0,11.0


###Writing custom formatters
Custom formatters can either be either added to pybloqs or included with user-space code. The latter is useful for very specific formatters, which have little chance of being reused and thus do not need to sit in the code base.
In general, formatters are classes that inherit from ```TableFormatter``` base class. The base class provides the following function hooks, which do not need to be all implemented by the new formatter. In fact, most formatters only make use of one or two function hooks. The available hooks are:
* ```_insert_additional_html```: Can be used to put HTML or JavaScript code in front of table.
* ```_modify_dataframe```: Access and potentially modify DataFrame before any other hook functions become active.
* ```_modify_cell_content```: Access and potentially modify cell value. Called for each cell separately. 
* ```create_table_level_css```: Insert CSS inline style on ```<table>``` level
* ```_create_thead_level_css```: Insert CSS inline style on ```<thead>``` level
* ```_create_row_level_css```: Insert CSS inline style on ```<th>``` and ```<tr>```level
* ```_create_cell_level_css```: Insert CSS inline style on ```<td>``` level
